학습? => 훈련 데이터셋으로부터 최적의 가중치 매개변수값을 획득하는 것.
- 학습 지표로 쓰이는 Loss Function!
- 따라서 학습 = 손실 함수 결괏값을 최소로 만드는 가중치 매개변수를 찾는 것.


머신러닝은 인간 중심? => ㄴㄴ 무조건 **데이터 주도 학습**.
- 사람의 직관, 경험등의 개입을 최소화하고 수집한 데이터들로부터 패턴을 찾으려 함.
- 다만 분류 task에 따라 적절한 특징 선정엔 사람이 관여. (SIFT, SURF..)

딥러닝은?? => 사람의 개입이 일절 없음.
- 종단간(처음부터 끝까지) 머신러닝이라고도 불리우는 이유.
- task(개/고양이 분류? 숫자 분류?)에 관계없이 모든 문제를 주어진 데이터를 온전히 학습하고 패턴을 탐색함.

데이터셋은 범용성 테스트를 위해 훈련용과 시험용으로 나눠야함.
- overfitting 피하기는 머신러닝의 중요한 과제


### 손실함수
- 최적의 매개변수 값을 탐색하는 **단 하나의 지표로 얼마나 오차가 나는가를 나타냄**. 
- 일반적으로 *오차제곱합*, *교차 엔트로피 오차*를 사용

#### 오차제곱합

$E = \frac{1}{2} \sum_{k} (y_k - t_k)^2$  

- 신경망의 출력과 정답 레이블 간의 차이를 제곱하여 모두 더한 후, 1/2를 곱한 값.
- 신경망의 출력이 정답 레이블과 얼마나 차이가 나는지를 측정 가능.

In [1]:
import numpy as np

y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0] # 2일 확률이 높다고 생각하는 신경망의 출력
t  = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0] # 정답은 2

In [2]:
def sum_squares_error(y, t): # 오차제곱합 구현
    return 0.5 * np.sum((y-t)**2)

In [3]:
sum_squares_error(np.array(y), np.array(t)) # 

0.09750000000000003

In [4]:
y2 = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0] # 7일 확률이 높다고 생각하는 신경망의 출력
sum_squares_error(np.array(y2), np.array(t))


0.5975

#### 교차 엔트로피 오차
$E = - \sum_{k} t_k \log y_k$  

- 이 때 t는 원-핫 인코딩 정답 레이블
- 따라서 위 식은 실질적으로 정답일 때, 즉 $t_k = 1$일 때  $-\log y_k$만을 계산한다.
- y_k는 항상 0~1이기 때문에 로그에 마이너스를 붙여준다.

In [5]:
def cross_entropy_error(y,t):
    delta = 1e-7  
    return -np.sum(t * np.log(y+delta)) # 입력값 0 방지를 위해 아주 작은 델타값 추가

In [6]:
print(cross_entropy_error(np.array(y), np.array(t)), cross_entropy_error(np.array(y2), np.array(t)))

0.510825457099338 2.302584092994546


즉 머신러닝은 훈련 데이터셋에 대한 **손실함수** 값을 구하고 이 값을 최대한 줄여주는 매개변수를 찾는 과정이다.

데이터 하나마다 손실함수를 구하여 모든 데이터에 대한 손실함수 값을 N으로 나누어 정규화해 **평균 손실함수**를 구한다!
- 근데 훈련 데이터셋이 MNIST처럼 60,000개, 혹은 그 이상의 빅데이터라면?
- 너무 많은 데이터 대상으로 일일히 손실함수를 계산하는건 비현실적.

### 미니배치 학습
- 훈련셋중 일부를 추려 전체의 '근사치'로 이용 => **미니배치**를 사용하여 학습하는 방법
- 이를테면 MNIST 훈련셋 6만개 중 배치 사이즈를 100으로 지정해 전체에서 일부를 선별해 근사치 값을 구한다.
- np.random.choice(훈련셋 크기, 배치 사이즈)로 미니배치를 지정할 랜덤 인덱스들을 뽑아낼 수 있다.

### 정확도 vs 손실함수
- 매개변수를 조정하는 기준이 정확도가 아니라 손실값인 이유?
- 정확도를 지표로 사용하면 매개변수의 미분이 대부분의 장소에서 0이 되기 때문이다!

In [9]:
# Gradient : 모든 변수의 편미분을 벡터로 정리한 것.

def numerical_gradient(f,x):
    h = 1e-4 # 극미량의 값은 반올림 오차가 없는 값을 선정해야하는데 1e-4가 좋음
    
    grad = np.zeros_like(x) # x와 형상이 같은 0초기화 배열 생성
    for i in range(x.size):
        tmp_val = x[i]

        x[i] = tmp_val + h
        fxh1 = f(x) # f(x+h)

        x[i] = tmp_val - h
        fxh2 = f(x) # f(x-h)

        grad[i] = (fxh1 - fxh2) / (2*h)
        x[i] = tmp_val
    
    return grad


In [10]:
def function_2(x):
    return x[0]**2 + x[1]**2

numerical_gradient(function_2, np.array([3.0,4.0])) # [6., 8.]
# y = x0^2 + x1^2의 점 (3,4)에서 기울기는 (6,8)을 의미

array([6., 8.])

## 경사 하강법
- 광대한 매개변수 공간에서 손실 함수값이 최소가 되는 지점을 직접 찾는 건 불가능
- 이때 기울기가 가리키는 쪽은 각 장소에서 함수의 출력값을 가장 크게 줄이는 방향! (gradient_2d.py 생각)
- 따라서 기울기를 잘 이용해 함수의 최솟값(혹은 가능한 한 작은 값)을 찾아보는 기법이 경사 하강법

But (한번만으론??)
- 기울기가 가리키는 곳이 함수의 최솟값이란 보장이 없음
- 극솟값 혹은 안장점일 수도.. 재수없으면 고원.
- 단지 그 방향으로 가야 함수의 값을 낮출 수 있다는 것



In [11]:
def gradient_descent(f, init_x, lr=0.01, step_num=100):
    x=init_x

    for i in range(step_num):
        grad = numerical_gradient(f,x)
        x = x - lr*grad
    
    return x

# 경사법으로 함수의 극솟값을 구할 수 있고 잘하면 최솟값을 구할 수도 있다

In [14]:
gradient_descent(function_2, init_x=np.array([-3.0,4.0]), lr=0.1)
# 사실상 0,0에 가까운 점이 나온다!

array([-6.11110793e-10,  8.14814391e-10])

In [16]:
lr_big_result = gradient_descent(function_2, init_x=np.array([-3.0,4.0]), lr=10.0)
lr_small_result = gradient_descent(function_2, init_x=np.array([-3.0,4.0]), lr=1e-10)

print(lr_big_result, lr_small_result)
# 학습률이 너무 크면 발산, 너무 작으면 init_x랑 별 차이 없음

[-2.58983747e+13 -1.29524862e+12] [-2.99999994  3.99999992]
